In [ ]:
# Giotto-TDA (includes ripser parallel)
# https://giotto-ai.github.io/gtda-docs/0.5.1/index.html
#
# scikit-tda (includes one-thread ripser)
# https://scikit-tda.org/
#
# Ripser++ (GPU)
# https://github.com/simonzhang00/ripser-plusplus
#
# Credits to: https://github.com/lizliz (Elizabeth Munch), https://github.com/giotto-ai/giotto-tda/tree/master/examples (EPFL)

In [ ]:
!pip install scikit-tda

In [ ]:
!pip install teaspoon

In [ ]:
# Basic imports 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import networkx as nx
#from IPython.display import Video

# scikit-tda imports..... Install all with -> pip install scikit-tda
#--- this is the main persistence computation workhorse
import ripser
# from persim import plot_diagrams
import persim
# import persim.plot

# teaspoon imports...... Install with -> pip install teaspoon
#---these are for generating data and some drawing tools 
import teaspoon.MakeData.PointCloud as makePtCloud
import teaspoon.TDA.Draw as Draw

#---these are for generating time series network examples
#from teaspoon.SP.network import ordinal_partition_graph
#from teaspoon.TDA.PHN import PH_network
#from teaspoon.SP.network_tools import make_network
#from teaspoon.parameter_selection.MsPE import MsPE_tau
#import teaspoon.MakeData.DynSysLib.DynSysLib as DSL

### Noisy Circle

In [ ]:
r = 1
R = 2
P = makePtCloud.Annulus(N=200, r=r, R=R, seed=None) # teaspoon data generation
plt.scatter(P[:,0],P[:,1])
# print(P)
# print(type(P))
# print(P.shape)

In [ ]:
# Some quick code to draw stuff without showing all the matplotlib junk in the slides everytime. 

def drawTDAtutorial(P,diagrams, R = 2):
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize = (20,5))

    # Draw point cloud 
    plt.sca(axes[0])
    plt.title('Point Cloud')
    plt.scatter(P[:,0],P[:,1])

    # Draw diagrams
    plt.sca(axes[1])
    plt.title('0-dim Diagram')
    Draw.drawDgm(diagrams[0])

    plt.sca(axes[2])
    plt.title('1-dim Diagram')
    Draw.drawDgm(diagrams[1])
    plt.axis([0,R,0,R])

In [ ]:
diagrams = ripser.ripser(P)['dgms']

# Draw stuff
drawTDAtutorial(P,diagrams)  # Script included in notebook for drawing

**Storage of diagrams**

In [ ]:
# Some discussion of how diagrams are stored 
data = ripser.ripser(P)
# print(data.keys())
# print(data['dgms'])
data['dgms'][1]
# len(data['dgms'])

### Random cube example

In [ ]:
P = makePtCloud.Cube()
diagrams = ripser.ripser(P)['dgms']

# Draw stuff
drawTDAtutorial(P,diagrams,R=0.8) # Script for drawing everything, code included in notebook

### Double noisy circle

In [ ]:
# Make a quick double circle

def DoubleCircle(r1 = 1, R1 = 2, r2 = .8, R2 = 1.3, xshift = 3):
    P = makePtCloud.Annulus(r = r1, R = R1)
    Q = makePtCloud.Annulus(r = r2, R = R2)
    Q[:,0] = Q[:,0] + xshift
    P = np.concatenate((P, Q) )
    return(P)

P = DoubleCircle(r1 = 1, R1 = 2, r2 = .5, R2 = 1.3, xshift = 3) 
plt.scatter(P[:,0], P[:,1])

In [ ]:
P = DoubleCircle(r1 = 1, R1 = 2, r2 = .5, R2 = 1.3, xshift = 3) # Code included in notebook
diagrams = ripser.ripser(P)['dgms']

# Draw stuff
drawTDAtutorial(P,diagrams,R=2.5) # Script for drawing everything, code included in notebook

### Torus

In [ ]:
!pip install tadasets

In [ ]:
import tadasets

#torus = tadasets.torus(n=500, c=2, a=1, ambient=100, noise=0.01)
torus = tadasets.torus(n=500, c=1, a=0.1, noise=0.01)

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(1, 1, 1, projection = '3d')
ax1.scatter(xs = torus[:, 0], ys = torus[:, 1], zs = torus[:, 2], s = 1, cmap = plt.cm.rainbow)

ax1.view_init(25, 10)

ax1.set_xlim(-3, 3)
ax1.set_ylim(-3, 3)
ax1.set_zlim(-3, 3)

In [ ]:
diagrams = ripser.ripser(torus, maxdim = 2)['dgms']

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize = (20,5))

# Draw diagrams
plt.sca(axes[0])
plt.title('0-dim Diagram')
Draw.drawDgm(diagrams[0])

plt.sca(axes[1])
plt.title('1-dim Diagram')
Draw.drawDgm(diagrams[1])

plt.sca(axes[2])
plt.title('2-dim Diagram')
Draw.drawDgm(diagrams[2])

### Distance between persistence diagrams

In [ ]:
# Make three example point clouds 
r = 1
R = 2
P1 = makePtCloud.Annulus(N=200, r=r, R=R, seed=None) # teaspoon data generation
P2 = makePtCloud.Annulus(N=200, r=r, R=R, seed=None)
P2[:,1] += 6
P3 = DoubleCircle()
P3 *= 1.1
P3[:,0] += 6
P3[:,1] += 3

In [ ]:
# plt.figure(figsize = (15,5))
plt.scatter(P1[:,0],P1[:,1], label = 'P1')
plt.scatter(P2[:,0],P2[:,1], label = 'P2')
plt.scatter(P3[:,0],P3[:,1], label = 'P3')
plt.axis('equal')
plt.legend()

In [ ]:
# Compute their diagrams 
diagrams1 = ripser.ripser(P1)['dgms']
diagrams2 = ripser.ripser(P2)['dgms']
diagrams3 = ripser.ripser(P3)['dgms']

Draw.drawDgm(diagrams1[1])
Draw.drawDgm(diagrams2[1])
Draw.drawDgm(diagrams3[1])

**Bottleneck distance**

In [ ]:
# Compute bottleneck distance using scikit-tda
distance_bottleneck, matching = persim.bottleneck(diagrams1[1], diagrams3[1], matching=True)
persim.visuals.bottleneck_matching(diagrams1[1], diagrams3[1], matching, labels=['Clean $H_1$', 'Noisy $H_1$'])
print('The bottleneck distance is', distance_bottleneck)
# print(matching)
# print(D)

In [ ]:
persim.bottleneck(diagrams1[1], diagrams2[1], matching=True)

### Computing Persistence on a Pairwise Distance/Similarity Matrix

For this tutorial, we will always use the clique complex, but there are other options available.

Some examples of when we might want to compute persistence in this way:

Input data with a distance/similarity matrix
* Weighted graph where we set distance between non adjacent vertices to be np.inf-
* Computing persistence for a weighted graph as the 1-skeleton

![Google's logo](https://raw.githubusercontent.com/IlyaTrofimov/TDA_Sirius2022/df4c6d5cd4cf29327e1875a885dd05695e31f913/WeightedGraphCliqueExample.png)

In [ ]:
# Generate the distance matrix from the previous example
D = np.array([[0, 1, np.inf, np.inf, 6],  [0, 0, 5, np.inf, np.inf],  [0, 0, 0, 2, 4],  [0, 0, 0, 0, 3],  [0, 0, 0, 0, 0]])
D = D+D.T
print(D)

In [ ]:
diagrams = ripser.ripser(D, distance_matrix=True, maxdim=1)['dgms']
print('0-Dim Diagram')
print(diagrams[0])
print('1-Dim Diagram')
print(diagrams[1])

**A bigger example with an Erdos-Renyii random graph**

In [ ]:
# Drawing script for weighted graph
def drawGraphEx(G):
    #draw it!

    pos = nx.spring_layout(G)  # positions for all nodes - seed for reproducibility

    # nodes
    nx.draw_networkx_nodes(G, pos, node_size=70)

    # edges
    nx.draw_networkx_edges(G, pos,  width=2)
    # nx.draw_networkx_edges(
    #     G, pos, edgelist=esmall, width=6, alpha=0.5, edge_color="b", style="dashed"
    # )

    # labels
    # nx.draw_networkx_labels(G, pos, font_size=20, font_family="sans-serif")
    edge_labels=nx.draw_networkx_edge_labels(G,pos,edge_labels=nx.get_edge_attributes(G, 'weight'))

In [ ]:
n = 10
p = .3

# Generate random graph 
G = nx.erdos_renyi_graph(n, p, seed=None, directed=False)

m = len(G.edges)
print('There are', m,'edges.')

# Generate random edge weights in the interval [0,maxWeight]
maxWeight = 100
weights = np.random.randint(maxWeight, size = m)

for i, e in enumerate(G.edges()):
    G[e[0]][e[1]] ['weight'] = weights[i]
    
drawGraphEx(G)

In [ ]:
A = nx.adjacency_matrix(G, weight = 'weight')
A = A.todense() # Turn into dense matrix for ease of messing with it
A = np.array(A) # Apparently I need to hand scikit-tda an array instead of a matrix, don't know why
A = A.astype('float64') # Needed to let me put in np.inf
A[ np.where(A == 0)] = np.inf
np.fill_diagonal(A,0)

im = plt.matshow(A, vmax = 100) # Note the np.inf values show up as white
plt.colorbar(im)

In [ ]:
diagrams = ripser.ripser(A, distance_matrix=True)['dgms']
persim.plot_diagrams(diagrams)
# print(diagrams)
# print(diagrams)

Application: graph classification

# Time-delay (Takens) Embedding

**Periodic example**

In [ ]:
import numpy as np
import plotly.graph_objects as go

x_periodic = np.linspace(0, 10, 1000)
y_periodic = np.cos(5 * x_periodic)

fig = go.Figure(data=go.Scatter(x=x_periodic, y=y_periodic))
fig.update_layout(xaxis_title="Timestamp", yaxis_title="Amplitude")
fig.show()

In [ ]:
!pip install giotto-tda

In [ ]:
from gtda.time_series import SingleTakensEmbedding

embedding_dimension_periodic = 3
embedding_time_delay_periodic = 8   # Time delay between two consecutive values for constructing one embedded point.
stride = 10                         # Stride duration between two consecutive embedded points

embedder_periodic = SingleTakensEmbedding(
    parameters_type="fixed",
    n_jobs=2,
    time_delay=embedding_time_delay_periodic,
    dimension=embedding_dimension_periodic,
    stride=stride,
)

In [ ]:
y_periodic_embedded = embedder_periodic.fit_transform(y_periodic)
print(f"Shape of embedded time series: {y_periodic_embedded.shape}")

In [ ]:
from gtda.plotting import plot_point_cloud

plot_point_cloud(y_periodic_embedded)

**Non-periodic example**

In [ ]:
x_nonperiodic = np.linspace(0, 50, 1000)
y_nonperiodic = np.cos(x_nonperiodic) + np.cos(np.pi * x_nonperiodic)

fig = go.Figure(data=go.Scatter(x=x_nonperiodic, y=y_nonperiodic))
fig.update_layout(xaxis_title="Timestamp", yaxis_title="Amplitude")
fig.show()

In [ ]:
embedding_dimension_nonperiodic = 3
embedding_time_delay_nonperiodic = 16
stride = 3

embedder_nonperiodic = SingleTakensEmbedding(
    parameters_type="fixed",
    n_jobs=2,
    time_delay=embedding_time_delay_nonperiodic,
    dimension=embedding_dimension_nonperiodic,
    stride=stride,
)

y_nonperiodic_embedded = embedder_nonperiodic.fit_transform(y_nonperiodic)

plot_point_cloud(y_nonperiodic_embedded)


In [ ]:
y_periodic_embedded = y_periodic_embedded[None, :, :]
y_nonperiodic_embedded = y_nonperiodic_embedded[None, :, :]

In [ ]:
from gtda.homology import VietorisRipsPersistence

# 0 - connected components, 1 - loops, 2 - voids
homology_dimensions = [0, 1, 2]

periodic_persistence = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, n_jobs=6
)
print("Persistence diagram for periodic signal")
periodic_persistence.fit_transform_plot(y_periodic_embedded)

nonperiodic_persistence = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, n_jobs=6
)
print("Persistence diagram for nonperiodic signal")
nonperiodic_persistence.fit_transform_plot(y_nonperiodic_embedded);

Applications of Takens embedding:
* time-series classification
* time-series forecasting
* meta-learning